In [71]:
import pandas as pd
import json
import numpy as np

# 读取包含数据的JSON文件
df = pd.read_json(r'../VeReMi-Dataset/ConstPosOffset_0709/VeReMi_25200_28800_2022-9-11_20.45.50/VeReMi_25200_28800_2022-9-11_20_45_50/traceGroundTruthJSON-7.json', lines=True)

In [72]:
df.head()

,type,sendTime,sender,senderPseudo,messageID,pos,pos_noise,spd,spd_noise,acl,acl_noise,hed,hed_noise
0,4,25202.602763,9,1092,4184,"[265.6497836207899, 45.99199764605675, 0.0]","[4.771972012856962, 4.575706671970706, 0.0]","[-0.05054182604486, 0.48911680623782805, 0.0]","[6.6503265915e-05, -0.000643553343453, 0.0]","[-0.249470372164231, 2.414430209110833, 0.0]","[0.0013822546815190002, 0.0006721980721500001,...","[-0.096104521290318, 0.9953712478204091, 0.0]","[18.62185606914561, 17.643546606006872, 0.0]"
1,4,25203.602763,9,1092,4194,"[265.43297053230134, 47.69669814624807, 0.0]","[4.596826441379185, 4.391555224378903, 0.0]","[-0.26960923589401103, 2.609092633012841, 0.0]","[0.00035474847325, -0.0034329075858650004, 0.0]","[-0.20670125611134102, 2.00059128259658, 0.0]","[0.00028824520733500004, 0.002789354242411, 0.0]","[-0.09145286751938901, 0.9958094059720871, 0.0]","[19.034311477145604, 14.273048388813764, 0.0]"
2,4,25204.602763,9,1092,4204,"[265.24450726709904, 51.317563380652, 0.0]","[4.8747635571499845, 4.889163596971624, 0.0]","[-0.46156831699633905, 4.466804632216011, 0.0]","[0.0006073337424150001, -0.005877179942134001,...","[-0.14888892327432302, 1.4411255323777261, 0.0]","[0.000252585269165, 0.0024442723562690003, 0.0]","[-0.09067483858685901, 0.9958805518972881, 0.0]","[19.403243706936372, 11.85326027651438, 0.0]"
3,4,25205.602763,9,1092,4214,"[264.6095967466214, 56.89890391563284, 0.0]","[4.695983465027813, 5.014286986384895, 0.0]","[-0.7222017726880201, 6.53420380691254, 0.0]","[0.000950310705695, -0.008597315207859, 0.0]","[-0.27191444546827903, 2.460329136091667, 0.0]","[0.000342976963279, 0.0027201352657250004, 0.0]","[-0.08544797859268201, 0.9963426333116651, 0.0]","[19.915677918586102, 9.63947550257849, 0.0]"
4,4,25206.602763,9,1092,4224,"[266.64368882510365, 64.87698877324128, 0.0]","[4.737223135625045, 4.930223558917946, 0.0]","[-1.183722182214897, 8.412407673698, 0.0]","[0.0015575918031500002, -0.011068529639086002,...","[-0.18651006827031202, 1.325423817489684, 0.0]","[0.000607281097454, 0.002471214431226, 0.0]","[-0.11501291265473501, 0.9933639966913811, 0.0]","[20.856422930941658, 7.988867175642302, 0.0]"


In [73]:
#依据sender分组
group = df.groupby('sender')
for name, groupdf in group:
    print(name,groupdf.shape)

    # 使用nunique()计算不同值的数量
    SumP = groupdf['senderPseudo'].nunique()
    groupdf['SumP']=SumP

    # 按照时间顺序排序
    groupdf.sort_values(by='sendTime', inplace=True)
    # 计算时间间隔
    groupdf['IntervalTime'] = groupdf['sendTime'] - groupdf['sendTime'].shift(1)
    # 第一行数据设为0
    groupdf['IntervalTime'].fillna(0, inplace=True)

    # 使用apply和lambda表达式去除z坐标
    groupdf['pos'] = groupdf['pos'].apply(lambda x: x[:2])
    groupdf['spd'] = groupdf['spd'].apply(lambda x: x[:2])
    groupdf['acl'] = groupdf['acl'].apply(lambda x: x[:2])
    groupdf['hed'] = groupdf['hed'].apply(lambda x: x[:2])

    # 选择特定的列
    selected_columns = ['SumP', 'IntervalTime','pos', 'spd', 'acl', 'hed']
    groupdf_selected = groupdf[selected_columns]

    # 序列小于10舍弃
    if len(groupdf_selected)<10:
        continue
    # 截断超过200行的部分
    elif len(groupdf_selected) > 200:
        groupdf_selected = groupdf_selected.head(200)  
     # 用0填充不足200行的部分
    else:
        fill_rows =200 - len(groupdf_selected)
        groupdf_selected=pd.concat([groupdf_selected, pd.DataFrame(0, index=range(fill_rows), columns=groupdf_selected.columns[:2])])
        groupdf_selected = pd.concat([groupdf_selected, pd.DataFrame(np.zeros(
            (fill_rows, 4)), columns=groupdf_selected.columns[['pos', 'spd', 'acl', 'hed']])], ignore_index=True)
    print(name, groupdf_selected.shape)

    break;
    

9 (91, 13)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
groupdf_selected

,SumP,IntervalTime,pos,spd,acl,hed
0,1,0.0,"[265.6497836207899, 45.99199764605675]","[-0.05054182604486, 0.48911680623782805]","[-0.249470372164231, 2.414430209110833]","[-0.096104521290318, 0.9953712478204091]"
1,1,1.0,"[265.43297053230134, 47.69669814624807]","[-0.26960923589401103, 2.609092633012841]","[-0.20670125611134102, 2.00059128259658]","[-0.09145286751938901, 0.9958094059720871]"
2,1,1.0,"[265.24450726709904, 51.317563380652]","[-0.46156831699633905, 4.466804632216011]","[-0.14888892327432302, 1.4411255323777261]","[-0.09067483858685901, 0.9958805518972881]"
3,1,1.0,"[264.6095967466214, 56.89890391563284]","[-0.7222017726880201, 6.53420380691254]","[-0.27191444546827903, 2.460329136091667]","[-0.08544797859268201, 0.9963426333116651]"
4,1,1.0,"[266.64368882510365, 64.87698877324128]","[-1.183722182214897, 8.412407673698]","[-0.18651006827031202, 1.325423817489684]","[-0.11501291265473501, 0.9933639966913811]"
...,...,...,...,...,...,...
104,0,0.0,NaN,NaN,NaN,NaN
105,0,0.0,NaN,NaN,NaN,NaN
106,0,0.0,NaN,NaN,NaN,NaN
107,0,0.0,NaN,NaN,NaN,NaN
